In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
from cvzone.HandTrackingModule import HandDetector

mpHands=mp.solutions.hands                                                   # initialize mediapipe 
hands=mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw=mp.solutions.drawing_utils
model=load_model('mp_hand_gesture')                                          # Load the gesture recognizer model
f=open('gesture.names', 'r')                                                   # Load class names
classNames=f.read().split('\n')
f.close()
print(classNames)
cap=cv2.VideoCapture(0)                                                       # Initialize the webcam
detector=HandDetector()
while True:
    status, photo=cap.read()
    x, y, c=photo.shape
    photo=cv2.flip(photo, 1)
    photorgb=cv2.cvtColor(photo, cv2.COLOR_BGR2RGB)
    result=hands.process(photorgb)                                              # Get hand landmark prediction
    print(result)
    className=''
    myhand=detector.findHands(photo)                                            # Hand Detection
    if myhand and myhand[0]:
        mylmlist=myhand[0][0]
        myfingerup=detector.fingersUp(mylmlist)
    if result.multi_hand_landmarks:                                             # post process the result
        landmarks=[]
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                #print(id, lm)
                lmx=int(lm.x * x)
                lmy=int(lm.y * y)
                landmarks.append([lmx, lmy])
            mpDraw.draw_landmarks(photo, handslms, mpHands.HAND_CONNECTIONS)    # Drawing landmarks on frames
            prediction = model.predict([landmarks])                             # Predict gesture
            #print(prediction)
            classID=np.argmax(prediction)
            className=classNames[classID]
    cv2.putText(photo, className, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (57,255,20), 2, cv2.LINE_AA)
    cv2.imshow("Hand Gesture Module", photo)                                            # Show the final output
    if cv2.waitKey(1)==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()